# Error analysis 1
- Identification of high-value false negatives (conversions that the model does not detect). What in common do they have?
- Identify clusters of leads with similar characteristics but different conversion behaviors, and what information can lead the algorithm to classify better.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, f1_score, precision_score, recall_score
import os
import warnings
warnings.filterwarnings('ignore')

# 1. Carregar dados do mesmo caminho usado originalmente
print("Carregando conjuntos de dados originais...")
data_path = "datasets/split/"
train_df = pd.read_csv(f"{data_path}train.csv")
val_df = pd.read_csv(f"{data_path}validation.csv")

print(f"Dados carregados - treino: {train_df.shape}, validação: {val_df.shape}")

# 2. Identificar colunas comuns entre os conjuntos
common_cols = set(train_df.columns).intersection(set(val_df.columns))
print(f"Encontradas {len(common_cols)} colunas em comum entre os conjuntos")

# 3. Usar apenas colunas em comum
train_df = train_df[list(common_cols)]
val_df = val_df[list(common_cols)]

# 4. Sanitizar nomes das colunas (simplificado)
print("Sanitizando nomes das colunas...")
# Aplicamos mesma transformação em ambos os conjuntos para garantir consistência
target_col = 'target'
feature_cols = [col for col in train_df.columns if col != target_col]

# 5. Converter colunas inteiras para float
print("Convertendo colunas inteiras para float...")
for col in train_df.columns:
    if pd.api.types.is_integer_dtype(train_df[col].dtype):
        train_df[col] = train_df[col].astype(float)
        val_df[col] = val_df[col].astype(float)

# 6. Criar cópias para X e y
X_train = train_df[feature_cols].copy()
y_train = train_df[target_col].copy()
X_val = val_df[feature_cols].copy()
y_val = val_df[target_col].copy()

print(f"Taxa de conversão - treino: {y_train.mean():.4f}, validação: {y_val.mean():.4f}")

# 7. Treinar um modelo fresh com os mesmos parâmetros
print("\nTreinando novo modelo RandomForest com parâmetros originais...")
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')
rf_model.fit(X_train, y_train)
best_threshold = 0.15  # valor do modelo original

# 8. Gerar previsões
print("Gerando previsões...")
y_pred_prob_val = rf_model.predict_proba(X_val)[:, 1]
y_pred_val = (y_pred_prob_val >= best_threshold).astype(int)

# 9. Criar diretório para salvar análises
results_dir = "error_analysis"
os.makedirs(results_dir, exist_ok=True)

# 10. Calcular e visualizar matriz de confusão
print("Analisando erros...")
cm = confusion_matrix(y_val, y_pred_val)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Não Converteu', 'Converteu'],
            yticklabels=['Não Converteu', 'Converteu'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.tight_layout()
plt.savefig(f"{results_dir}/confusion_matrix.png")
plt.close()

# 11. Calcular métricas
tn, fp, fn, tp = cm.ravel()
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

print(f"\nMétricas de desempenho:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Taxa de Falsos Positivos: {fpr:.4f}")
print(f"Falsos Negativos: {fn} de {fn+tp} conversões reais")
print(f"Falsos Positivos: {fp} de {fp+tn} não-conversões reais")

# 12. Analisar distribuição de probabilidades
plt.figure(figsize=(10, 6))
plt.hist(y_pred_prob_val[y_val == 0], bins=50, alpha=0.5, color='blue', label='Não Converteu')
plt.hist(y_pred_prob_val[y_val == 1], bins=50, alpha=0.5, color='red', label='Converteu')
plt.axvline(x=best_threshold, color='green', linestyle='--', label=f'Threshold: {best_threshold:.4f}')
plt.title('Distribuição de Probabilidades Previstas')
plt.xlabel('Probabilidade')
plt.ylabel('Frequência')
plt.legend()
plt.savefig(f"{results_dir}/probability_distribution.png")
plt.close()

# 13. Análise de Falsos Negativos (conversões não detectadas)
false_negatives = (y_val == 1) & (y_pred_val == 0)
fn_indices = np.where(false_negatives)[0]
fn_data = val_df.iloc[fn_indices].copy()
fn_data['predicted_prob'] = y_pred_prob_val[fn_indices]

# Ordenar por probabilidade (do maior para o menor - casos limítrofes)
fn_data = fn_data.sort_values('predicted_prob', ascending=False)

# Salvar para análise detalhada
fn_data.to_csv(f"{results_dir}/false_negatives.csv", index=False)
print(f"\nNúmero de falsos negativos: {len(fn_data)}")
print(f"Lista detalhada salva em: {results_dir}/false_negatives.csv")

# 14. Identificar características comuns em falsos negativos de alto valor
print("\n--- Análise de Falsos Negativos de Alto Valor ---")
if len(fn_data) > 0:
    # Tentar identificar colunas numéricas e categóricas
    numeric_cols = []
    categorical_cols = []
    
    for col in val_df.columns:
        if col not in [target_col, 'predicted_prob'] and col in fn_data.columns:
            try:
                if pd.api.types.is_numeric_dtype(val_df[col]) and val_df[col].nunique() > 10:
                    numeric_cols.append(col)
                else:
                    categorical_cols.append(col)
            except:
                categorical_cols.append(col)
    
    # Limitar o número de colunas para análise
    numeric_cols = numeric_cols[:20]
    categorical_cols = categorical_cols[:20]
    
    # Análise de características numéricas
    if numeric_cols:
        print("\nCaracterísticas numéricas distintivas em falsos negativos:")
        numeric_analysis = []
        
        for col in numeric_cols:
            try:
                # Calcular estatísticas
                fn_mean = fn_data[col].mean()
                pop_mean = val_df[col].mean()
                
                # Calcular diferença percentual
                diff_pct = ((fn_mean - pop_mean) / pop_mean * 100) if pop_mean != 0 else 0
                
                numeric_analysis.append({
                    'Feature': col,
                    'FN_Mean': fn_mean,
                    'Population_Mean': pop_mean,
                    'Diff_Pct': diff_pct
                })
            except Exception as e:
                pass  # Ignorar erros silenciosamente
        
        # Ordenar por diferença percentual
        numeric_df = pd.DataFrame(numeric_analysis)
        if not numeric_df.empty:
            numeric_df = numeric_df.sort_values('Diff_Pct', ascending=False)
            
            # Mostrar top 5 características numéricas distintivas
            for i, row in numeric_df.head(5).iterrows():
                direction = "maior" if row['Diff_Pct'] > 0 else "menor"
                print(f"  - {row['Feature']}: {abs(row['Diff_Pct']):.2f}% {direction} que a média geral")
            
            # Salvar análise completa
            numeric_df.to_csv(f"{results_dir}/fn_numeric_features.csv", index=False)
    
    # Análise de características categóricas
    if categorical_cols:
        print("\nCaracterísticas categóricas distintivas em falsos negativos:")
        categorical_analysis = []
        
        for col in categorical_cols:
            try:
                # Calcular frequências
                pop_freq = val_df[col].value_counts(normalize=True).to_dict()
                fn_freq = fn_data[col].value_counts(normalize=True).to_dict()
                
                # Identificar valores com maior diferença
                for val, freq in fn_freq.items():
                    if val in pop_freq:
                        diff = freq - pop_freq[val]
                        ratio = freq / pop_freq[val] if pop_freq[val] > 0 else float('inf')
                        
                        if abs(diff) > 0.05:  # Apenas diferenças significativas
                            categorical_analysis.append({
                                'Feature': col,
                                'Value': val,
                                'FN_Freq': freq,
                                'Pop_Freq': pop_freq[val],
                                'Difference': diff,
                                'Ratio': ratio
                            })
            except Exception as e:
                pass  # Ignorar erros silenciosamente
        
        # Ordenar por maior diferença
        cat_df = pd.DataFrame(categorical_analysis)
        if not cat_df.empty:
            cat_df = cat_df.sort_values('Difference', ascending=False)
            
            # Mostrar top 5 características categóricas distintivas
            for i, row in cat_df.head(5).iterrows():
                print(f"  - {row['Feature']} = {row['Value']}: {row['FN_Freq']:.2%} (vs {row['Pop_Freq']:.2%} na população), {row['Ratio']:.2f}x mais comum")
            
            # Salvar análise completa
            cat_df.to_csv(f"{results_dir}/fn_categorical_features.csv", index=False)
    
    # 15. Análise por segmentos (tipo de lançamento, país, etc.)
    segment_cols = []
    
    # Identificar colunas de segmentação potencialmente interessantes
    for pattern in ['launch', 'lançament', 'country', 'pais', 'age', 'idade']:
        cols = [col for col in val_df.columns if pattern.lower() in col.lower()]
        if cols:
            segment_cols.extend(cols[:1])  # Adicionar apenas a primeira coluna encontrada para cada padrão
    
    # Limitar o número total de segmentos
    segment_cols = segment_cols[:5]
    
    if segment_cols:
        print("\nAnálise de falsos negativos por segmentos:")
        
        for col in segment_cols:
            if col in val_df.columns:
                try:
                    # Calcular taxa de falsos negativos por segmento
                    segment_stats = []
                    
                    # Limitar a 10 valores mais frequentes para evitar análise excessiva
                    top_values = val_df[col].value_counts().nlargest(10).index
                    
                    for value in top_values:
                        # Filtrar dados para este segmento
                        segment_mask = val_df[col] == value
                        segment_y_true = y_val[segment_mask]
                        segment_y_pred = y_pred_val[segment_mask]
                        
                        # Se houver dados suficientes
                        if sum(segment_mask) >= 20 and sum(segment_y_true) > 0:
                            # Calcular taxa de falsos negativos
                            segment_fn = ((segment_y_true == 1) & (segment_y_pred == 0)).sum()
                            segment_fn_rate = segment_fn / sum(segment_y_true)
                            
                            segment_stats.append({
                                'Segmento': value,
                                'Tamanho': sum(segment_mask),
                                'Conversões': sum(segment_y_true),
                                'Falsos_Negativos': segment_fn,
                                'Taxa_FN': segment_fn_rate
                            })
                    
                    # Ordenar por taxa de falsos negativos
                    segment_df = pd.DataFrame(segment_stats)
                    if not segment_df.empty:
                        segment_df = segment_df.sort_values('Taxa_FN', ascending=False)
                        segment_df.to_csv(f"{results_dir}/segment_analysis_{col}.csv", index=False)
                        
                        print(f"\nSegmentos com maior taxa de falsos negativos para {col}:")
                        for i, row in segment_df.head(3).iterrows():
                            print(f"  - {col}={row['Segmento']}: {row['Taxa_FN']:.2%} das conversões não detectadas ({row['Falsos_Negativos']} de {row['Conversões']})")
                except Exception as e:
                    pass  # Ignorar erros silenciosamente
    
    # 16. Comparação de perfil: conversões detectadas vs não detectadas
    print("\n--- Perfil Comparativo: Conversões Detectadas vs Não Detectadas ---")
    true_positives = (y_val == 1) & (y_pred_val == 1)
    tp_indices = np.where(true_positives)[0]
    tp_data = val_df.iloc[tp_indices].copy()
    
    # Selecionar features importantes para comparação
    top_features = []
    try:
        feature_imp = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': rf_model.feature_importances_
        })
        top_features = feature_imp.sort_values('Importance', ascending=False).head(10)['Feature'].tolist()
    except:
        # Se não conseguir obter importância de features, usar as numéricas
        top_features = numeric_cols[:10]
    
    # Comparar médias entre TPs e FNs
    comparison = []
    for col in top_features:
        if col in tp_data.columns and col in fn_data.columns:
            try:
                tp_mean = tp_data[col].mean()
                fn_mean = fn_data[col].mean()
                diff = tp_mean - fn_mean
                
                comparison.append({
                    'Feature': col,
                    'Detected_Mean': tp_mean,
                    'Missed_Mean': fn_mean,
                    'Difference': diff
                })
            except Exception as e:
                pass  # Ignorar erros silenciosamente
    
    # Mostrar diferenças mais significativas
    comp_df = pd.DataFrame(comparison)
    if not comp_df.empty:
        comp_df = comp_df.sort_values('Difference', ascending=False)
        comp_df.to_csv(f"{results_dir}/detected_vs_missed_comparison.csv", index=False)
        
        print("Principais diferenças entre conversões detectadas e não detectadas:")
        for i, row in comp_df.head(5).iterrows():
            print(f"  - {row['Feature']}: {row['Detected_Mean']:.4f} nas detectadas vs {row['Missed_Mean']:.4f} nas não detectadas")
else:
    print("Não há falsos negativos suficientes para análise detalhada.")

# 17. Análise de Falsos Positivos
false_positives = (y_val == 0) & (y_pred_val == 1)
fp_indices = np.where(false_positives)[0]
fp_data = val_df.iloc[fp_indices].copy()
fp_data['predicted_prob'] = y_pred_prob_val[fp_indices]

# Ordenar por probabilidade (do maior para o menor)
fp_data = fp_data.sort_values('predicted_prob', ascending=False)

# Salvar para análise detalhada
fp_data.to_csv(f"{results_dir}/false_positives.csv", index=False)
print(f"\nNúmero de falsos positivos: {len(fp_data)}")

# 18. Importância das features
plt.figure(figsize=(12, 8))
feature_imp = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
})
feature_imp = feature_imp.sort_values('Importance', ascending=False).head(30)
sns.barplot(x='Importance', y='Feature', data=feature_imp)
plt.title('Top 30 Features (Importância do RandomForest)')
plt.tight_layout()
plt.savefig(f"{results_dir}/feature_importance.png")
plt.close()
feature_imp.to_csv(f"{results_dir}/feature_importance.csv", index=False)

# 19. Resumo final
print("\n=== RESUMO DA ANÁLISE DE ERROS ===")
print(f"Total de registros analisados: {len(y_val)}")
print(f"Conversões reais: {y_val.sum()} ({y_val.mean():.2%})")
print(f"Conversões previstas: {y_pred_val.sum()} ({y_pred_val.mean():.2%})")
print(f"Falsos negativos: {fn} ({fn/(fn+tp):.2%} das conversões reais)")
print(f"Falsos positivos: {fp} ({fp/(fp+tn):.2%} das não-conversões reais)")
print(f"Precisão: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
print(f"Todos os resultados foram salvos em: {results_dir}/")

Carregando conjuntos de dados originais...
Dados carregados - treino: (74629, 225), validação: (15992, 225)
Encontradas 225 colunas em comum entre os conjuntos
Sanitizando nomes das colunas...
Convertendo colunas inteiras para float...
Taxa de conversão - treino: 0.0176, validação: 0.0176

Treinando novo modelo RandomForest com parâmetros originais...
Gerando previsões...
Analisando erros...

Métricas de desempenho:
Precision: 0.6139
Recall: 0.2199
F1 Score: 0.3238
Taxa de Falsos Positivos: 0.0025
Falsos Negativos: 220 de 282 conversões reais
Falsos Positivos: 39 de 15710 não-conversões reais

Número de falsos negativos: 220
Lista detalhada salva em: error_analysis/false_negatives.csv

--- Análise de Falsos Negativos de Alto Valor ---

Características numéricas distintivas em falsos negativos:
  - Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?_tfidf_conocer: 156.47% maior que a média geral
  - Déjame un mensaje_tfidf_deseo: 123.59% ma